In [ ]:
from bs4 import BeautifulSoup #for HTML parsing (decoding the grid structure from the website)
from selenium import webdriver # Using Selenium to interact with the webbrowser (get HTML, click cells, etc.)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from time import time 
from copy import deepcopy

def initialize_permanent_values(temp_grid, grid_colors):
    """
    First, we analyze the initial grid and color structure to determine permanent constraints:
    - Place permanent queens (1s) where mandatory.
    - Mark permanent zero (0) positions where queens cannot be placed.
    - In most cases this will not fully solve the board but it will narrow down the possibilities for higher algorithmic efficiency later on

    Args:
    - temp_grid: 2D list representing the current state of the grid (None for unfilled cells).
    - grid_colors: 2D list indicating the color of each cell.

    Returns:
    - Updated temp_grid with permanent constraints that apply
    - A set of permanent_ones (fixed queens' positions).
    - A set of permanent_zeros (positions where queens are disallowed).
    """

    rows, cols = len(temp_grid), len(temp_grid[0])
    permanent_ones = set()
    permanent_zeros = set()

    # Iterate over each row and column to apply rules
    for r in range(rows):
        row_values = [temp_grid[r][c] for c in range(cols)]
        if row_values.count(1) == cols:  # Row is fully 1
            for x in range(rows):
                for y in range(cols):
                    if grid_colors[x][y] == grid_colors[r][c]:
                        temp_grid[x][y] = 0
                        permanent_zeros.add((x, y))
                        print_temp_grid(temp_grid)

    # Same for columns
    for c in range(cols):
        col_values = [temp_grid[r][c] for r in range(rows)]
        if col_values.count(1) == rows:  # Column is fully 1
            for x in range(rows):
                for y in range(cols):
                    if grid_colors[x][y] == grid_colors[r][c]:
                        temp_grid[x][y] = 0
                        permanent_zeros.add((x, y))
                        print_temp_grid(temp_grid)

    # Apply rules for colors in a similar way
    unique_colors = set(color for row in grid_colors for color in row)
    for color in unique_colors:
        color_cells = [(r, c) for r in range(rows) for c in range(cols) if grid_colors[r][c] == color]
        if len(color_cells) == 1:  # If only one cell exists for a color
            r, c = color_cells[0]
            temp_grid[r][c] = 1 # Place a queen in the only valid position
            permanent_ones.add((r, c)) 
            print_temp_grid(temp_grid)
        elif all(temp_grid[r][c] == 1 for r, c in color_cells): # If color is fully 1
            for r, c in color_cells:
                temp_grid[r][c] = 0 # Mark other cells as 0
                permanent_zeros.add((r, c))
                print_temp_grid(temp_grid)

    # Apply rules for exclusive color dominance in rows and columns
    updated = True
    while updated:
        updated = False # Reset update flag to iterate until there is no change anymore (because blocking cells as 0, might help to block further cells in next step)

        # Check rows for exclusive color dominance
        for r in range(rows):
            if any(temp_grid[r][c] == 1 for c in range(cols)):  # Skip rows with a queen
                continue

            row_colors = [grid_colors[r][c] for c in range(cols) if temp_grid[r][c] is None]
            unique_colors = set(row_colors)
            if len(unique_colors) == 1 and row_colors:  # All blanks in the row are of one color
                exclusive_color = unique_colors.pop()
                print(f"Row {r} is dominated by color {exclusive_color}.")
                for x in range(rows):
                    for y in range(cols):
                        if grid_colors[x][y] == exclusive_color and x != r and temp_grid[x][y] is None:
                            temp_grid[x][y] = 0 # Set all cells of that color that are not in this row to 0
                            permanent_zeros.add((x, y))
                            updated = True

        # Check columns for exclusive color dominance
        for c in range(cols):
            if any(temp_grid[r][c] == 1 for r in range(rows)):  # Skip columns with a queen
                continue

            col_colors = [grid_colors[r][c] for r in range(rows) if temp_grid[r][c] is None]
            unique_colors = set(col_colors)
            if len(unique_colors) == 1 and col_colors:  # All blanks in the column are of one color
                exclusive_color = unique_colors.pop()
                print(f"Column {c} is dominated by color {exclusive_color}.")
                for x in range(rows):
                    for y in range(cols):
                        if grid_colors[x][y] == exclusive_color and y != c and temp_grid[x][y] is None:
                            temp_grid[x][y] = 0 # Set all cells of that color that are not in this column to 0
                            permanent_zeros.add((x, y))
                            updated = True

        # Check if all cells of a color are confined to one row or column
        for color in set(color for row in grid_colors for color in row):
            color_cells = [(r, c) for r in range(rows) for c in range(cols) if grid_colors[r][c] == color and temp_grid[r][c] is None]
            if color_cells:
                rows_involved = {r for r, c in color_cells}
                cols_involved = {c for r, c in color_cells}

                if len(rows_involved) == 1:  # All cells in one row
                    row = next(iter(rows_involved))
                    if any(temp_grid[row][c] == 1 for c in range(cols)):  # Skip if row has a queen
                        continue
                    print(f"Color {color} is confined to row {row}.")
                    for c in range(cols):
                        if (row, c) not in color_cells and temp_grid[row][c] is None:
                            temp_grid[row][c] = 0 # Set all row cells that have a different color to 0 (because it MUST be this color)
                            permanent_zeros.add((row, c))
                            updated = True

                elif len(cols_involved) == 1:  # All cells in one column
                    col = next(iter(cols_involved))
                    if any(temp_grid[r][col] == 1 for r in range(rows)):  # Skip if column has a queen
                        continue
                    print(f"Color {color} is confined to column {col}.")
                    for r in range(rows):
                        if (r, col) not in color_cells and temp_grid[r][col] is None:
                            temp_grid[r][col] = 0 # Set all column cells that have a different color to 0 (because it MUST be this color)
                            permanent_zeros.add((r, col))
                            updated = True

    # Further constraint propagation (set 0s) for placed queens
    for r in range(rows):
        for c in range(cols):
            if temp_grid[r][c] == 1:  # Only propagate constraints for placed 1s
                propagate_constraints(temp_grid, r, c, None, grid_colors, permanent_ones, permanent_zeros)
                print(f"Propagating constraints from cell {(r, c)}.")

    return temp_grid, permanent_ones, permanent_zeros

# Algorithm
def solve_with_backtracking_and_propagation(temp_grid, grid_colors, variables, permanent_ones=None, permanent_zeros=None):
    """
    Solves the puzzle using a combination of optimized backtracking and constraint propagation
    - places a temporary queen at the most constrained cells (least available options/ highest probability for a queen)
    - iteratively checks constraints and continues to place temp queens until either the solution is found or an invalid state occurs
    - if invalid state: Backtrack only the last placed temporary queen and try all other alternatives in the current state
    - should all options of this state be exhausted, backtrack second-to-last temp queen to explore new sub-path
    - if all options in one path are fully scouted, place a permanent 0 at the coordinates of the first placed temp queen
    - explore a new path. Since this is a deterministic puzzle, this will find the correct solution in milliseconds

    New Args:
    - r, c: Row and column indices of the placed queen.
    - variables: List of variables for constraints (unused in this implementation).
    - temporary: Boolean indicating if the propagation is temporary (can be undone).

    Returns:
    - The solved grid
    """
    
    # initialize permanent constraints
    if permanent_ones is None:
        permanent_ones = set()
    if permanent_zeros is None:
        permanent_zeros = set()

    # backtracking stacks to exclude options if they've already been tried (otherwise we'd stuck in an infite loop, retrying the same option again)
    rows, cols = len(temp_grid), len(temp_grid[0])
    backtracking_stack = []  # Stack to track placed temporary queens and alternative options 
    first_temp_queen = None  # Track the first temporary queen in the current path
    tried_first_queen = set()  # Keep track of all first queens already tried
    tried_temp_queens = {}   # Map board states to queens that have already been tried

    # Step 1: Initialize permanent values based on rules (the function from above)
    temp_grid, permanent_ones, permanent_zeros = initialize_permanent_values(temp_grid, grid_colors)

    # Simplify the grid if possible
    global_temp_zeros = place_temp_queens(temp_grid, grid_colors, variables)
    print("\nAfter applying constraints to the grid:")
    print_temp_grid(temp_grid)

    print("\nStarting backtracking with most constrained cell heuristic...")

    while True:
        # Generate a unique representation of the current board state
        board_state = tuple(tuple(row) for row in temp_grid)

        # Step 2: Validate the current grid state
        if not is_valid_state(temp_grid, grid_colors):
            # If the state is invalid, backtrack to a previous decision point
            if not backtracking_stack:
                print("Stack is empty after exhaustive backtracking")
                if first_temp_queen:
                    # Mark the first queen placement as invalid (permanent zero)
                    r, c = first_temp_queen
                    temp_grid[r][c] = 0
                    permanent_zeros.add((r, c))
                    tried_first_queen.add(first_temp_queen)
                    first_temp_queen = None
                else:
                    print("No solution found.")
                    return None

            # Backtrack: Remove the last placed queen
            last_queen, tried_options, temp_zeros = backtracking_stack.pop()
            r, c = last_queen
            temp_grid[r][c] = None

            # Reset temporary zeros placed as a result of this temp queen
            for tr, tc in temp_zeros:
                temp_grid[tr][tc] = None

            print(f"Backtracking: Removed queen at ({r}, {c}). Updated grid:")
            print_temp_grid(temp_grid)

            reset_temp_queens_and_zeros(temp_grid, {last_queen}, global_temp_zeros, permanent_ones, permanent_zeros)

            # Update the block list for the current board state
            tried_temp_queens[board_state] = tried_temp_queens.get(board_state, set())
            tried_temp_queens[board_state].add((r, c))

            # If we backtracked to the first temporary queen and exhausted all options, set it as a permanent zero
            if first_temp_queen and (r, c) == first_temp_queen:
                temp_grid[r][c] = 0
                permanent_zeros.add((r, c))
                tried_first_queen.add(first_temp_queen)
                first_temp_queen = None

            continue

        # Step 3: Check if solution is complete
        if is_goal_state(temp_grid):
            return temp_grid

        # Step 4: Find the most constrained cell to prioritise it
        most_constrained_cells = find_most_constrained_cells(temp_grid, grid_colors)
        print(f"Most constrained cells: {most_constrained_cells}")
        next_cell = None

        # Check the most constrained cells for an untried option (exclude options we already tried)
        for cell in most_constrained_cells:
            if first_temp_queen is None and cell in tried_first_queen:
                continue

            if cell not in tried_temp_queens.get(board_state, set()) and not any(cell == queen for queen, _, _ in backtracking_stack):
                next_cell = cell
                break

        if not next_cell:
            print("No valid cell to place a queen. Backtracking...")
            if not backtracking_stack:
                print("No solution found: Backtracking stack is empty")
                return None

            # Backtrack: Remove last placed queen
            last_queen, tried_options, temp_zeros = backtracking_stack.pop()
            r, c = last_queen
            temp_grid[r][c] = None

            # Reset temporary zeros placed as a result of this queen
            for tr, tc in temp_zeros:
                temp_grid[tr][tc] = None

            print(f"Backtracking: Removed queen at ({r}, {c}). Updated grid:")
            print_temp_grid(temp_grid)

            reset_temp_queens_and_zeros(temp_grid, {last_queen}, global_temp_zeros, permanent_ones, permanent_zeros)

            # Update the block list for the current board state
            tried_temp_queens[board_state] = tried_temp_queens.get(board_state, set())
            tried_temp_queens[board_state].add((r, c))

            continue

        # Step 5: Place a temporary queen and propagate constraints
        r, c = next_cell
        tried_temp_queens[board_state] = tried_temp_queens.get(board_state, set())
        tried_temp_queens[board_state].add((r, c))
        if first_temp_queen is None:
            first_temp_queen = (r, c)
        temp_grid[r][c] = 1  # Temporarily place a queen at the most constrained & untried cell
        temp_zeros = propagate_constraints(temp_grid, r, c, variables, grid_colors, permanent_ones, permanent_zeros, temporary=True)

        # Save state to backtracking stack as temporary
        backtracking_stack.append((next_cell, set(tried_temp_queens[board_state]), temp_zeros))

        print(f"Placed temporary queen at ({r}, {c}). Current board state:")
        print_temp_grid(temp_grid)

def propagate_constraints(temp_grid, r, c, variables, grid_colors, permanent_ones, permanent_zeros, temporary=False):
    """
    Propagates constraints (place zeros) based on a newly placed queen to reduce search space.

    Returns:
    - A set of positions that were marked as temporary zeros during propagation.
    """

    rows, cols = len(temp_grid), len(temp_grid[0])
    temp_zeros = set() # Track all temporary zeros introduced during this propagation

    # Mark all cells in the same row, column as 0
    for i in range(rows):
        if temp_grid[i][c] is None:  # If cell is unfilled
            temp_grid[i][c] = 0
            if temporary:
                temp_zeros.add((i, c))
    for j in range(cols):
        if temp_grid[r][j] is None:  
            temp_grid[r][j] = 0
            if temporary:
                temp_zeros.add((r, j))

    # Mark immediate diagonal neighbors as 0
    for dr, dc in [(-1, -1), (-1, 1), (1, -1), (1, 1)]:
        nr, nc = r + dr, c + dc
        if 0 <= nr < rows and 0 <= nc < cols:  # Ensure within bounds
            if temp_grid[nr][nc] is None:
                temp_grid[nr][nc] = 0
                if temporary:
                    temp_zeros.add((nr, nc))

    # Mark all cells of the same color as zeros
    current_color = grid_colors[r][c]
    for i in range(rows):
        for j in range(cols):
            if grid_colors[i][j] == current_color and temp_grid[i][j] is None:
                temp_grid[i][j] = 0
                if temporary:
                    temp_zeros.add((i, j))

    # Block rows or columns if all remaining cells of a color are within them
    color_cell_positions = {}
    for i in range(rows):
        for j in range(cols):
            color = grid_colors[i][j]
            if temp_grid[i][j] is None:
                if color not in color_cell_positions:
                    color_cell_positions[color] = []
                color_cell_positions[color].append((i, j))

    for color, positions in color_cell_positions.items():
        if len(positions) > 1:
            rows_involved = {pos[0] for pos in positions}
            cols_involved = {pos[1] for pos in positions}
            if len(rows_involved) == 1:  # All cells in the same row
                row = next(iter(rows_involved))
                for col in range(cols):
                    if (row, col) not in positions and temp_grid[row][col] is None: #cells that are NOT the exclusive color
                        temp_grid[row][col] = 0 
                        temp_zeros.add((row, col))
            if len(cols_involved) == 1:  # All cells in the same column
                col = next(iter(cols_involved))
                for row in range(rows):
                    if (row, col) not in positions and temp_grid[row][col] is None:
                        temp_grid[row][col] = 0
                        temp_zeros.add((row, col))

    # Check rows and columns for 100% blank cells of one color 
    def check_and_update_color_constraints():
        updated = True
        while updated:
            updated = False
            for i in range(rows):
                row_colors = [grid_colors[i][j] for j in range(cols) if temp_grid[i][j] is None]
                if len(set(row_colors)) == 1 and row_colors:  # All blanks in the row are of one color
                    exclusive_color = row_colors[0]
                    for x in range(rows):
                        for y in range(cols):
                            if grid_colors[x][y] == exclusive_color and x != i and temp_grid[x][y] is None:
                                temp_grid[x][y] = 0 # Mark as 0 all other cells of that color (that are not in this row)
                                temp_zeros.add((x, y))
                                updated = True

            for j in range(cols):
                col_colors = [grid_colors[i][j] for i in range(rows) if temp_grid[i][j] is None]
                if len(set(col_colors)) == 1 and col_colors:  # All blanks in the column are of one color
                    exclusive_color = col_colors[0]
                    for x in range(rows):
                        for y in range(cols):
                            if grid_colors[x][y] == exclusive_color and y != j and temp_grid[x][y] is None:
                                temp_grid[x][y] = 0 
                                temp_zeros.add((x, y))
                                updated = True

    check_and_update_color_constraints() # reiterate if a change has been made

    return temp_zeros


def place_temp_queens(temp_grid, grid_colors, variables):
    """
    - set temp queens and temp zeros
    - returns a set of temporary zeros placed during the propagation (to allow backtracking)
    """

    rows, cols = len(temp_grid), len(temp_grid[0])
    temp_zeros = set()
    
    # Ensure one queen per color group
    for color in set(grid_colors[r][c] for r in range(rows) for c in range(cols)):
        color_cells = [
            (r, c) for r in range(rows) for c in range(cols)
            if grid_colors[r][c] == color and temp_grid[r][c] is None
        ]
        if len(color_cells) == 1:  # Only one valid cell for this color
            r, c = color_cells[0]
            temp_grid[r][c] = 1 #Place temp queen
            temp_zeros.update(propagate_constraints(temp_grid, r, c, variables, grid_colors, set(), set(), temporary=True))

    # Ensure one queen per row
    for r in range(rows):
        row_cells = [(r, c) for c in range(cols) if temp_grid[r][c] is None]
        if len(row_cells) == 1:  # Only one valid cell for this row
            _, c = row_cells[0]
            temp_grid[r][c] = 1  
            temp_zeros.update(propagate_constraints(temp_grid, r, c, variables, grid_colors, set(), set(), temporary=True))

    # Ensure one queen per column
    for c in range(cols):
        col_cells = [(r, c) for r in range(rows) if temp_grid[r][c] is None]
        if len(col_cells) == 1:  # Only one valid cell for this column
            r, _ = col_cells[0]
            temp_grid[r][c] = 1  
            temp_zeros.update(propagate_constraints(temp_grid, r, c, variables, grid_colors, set(), set(), temporary=True))

    return temp_zeros


def reset_temp_queens_and_zeros(temp_grid, temp_queens, temp_zeros, permanent_ones, permanent_zeros):
    """
    Reset temporary queens (1s) and their associated temp zeros to None.
    Permanently placed 1s and 0s remain unchanged.
    
    """
    reset_ones = []  # Track temporary queens that are reset
    reset_zeros = []  # Track temporary zeros that are reset

    # Reset temporary queens (1s)
    for r, c in temp_queens:
        if (r, c) not in permanent_ones:  # Only reset temporary queens
            temp_grid[r][c] = None
            reset_ones.append((r, c))

    # Reset temporary zeros (0s)
    for r, c in temp_zeros:
        if (r, c) not in permanent_zeros:  # Only reset temporary zeros
            temp_grid[r][c] = None
            reset_zeros.append((r, c))

    # Debugging output
    print("\nReset all temporary queens and zeros:")
    print("Temporary Queens Reset:", reset_ones)
    print("Temporary Zeros Reset:", reset_zeros)
    print_temp_grid(temp_grid)

def is_valid_state(temp_grid, grid_colors):
    """
    - Checks if the current grid state is valid based on game rules (constraints).
    - Returns "True" if the state is valid; "False" otherwise.
    """

    rows, cols = len(temp_grid), len(temp_grid[0])

    # Rule 1: One queen per row
    for r in range(rows):
        if sum(cell == 1 for cell in temp_grid[r]) > 1:
            print(f"Invalid state: More than one queen in row {r}. Row: {temp_grid[r]}")
            return False

    # Rule 2: One queen per column
    for c in range(cols):
        column = [temp_grid[r][c] for r in range(rows)]  # Extract the column
        if sum(cell == 1 for cell in column) > 1:
            print(f"Invalid state: More than one queen in column {c}. Column: {column}")
            return False

    # Rule 3: One queen per color
    color_groups = {}
    for r in range(rows):
        for c in range(cols):
            color = grid_colors[r][c]
            if color not in color_groups:
                color_groups[color] = []
            if temp_grid[r][c] == 1:
                color_groups[color].append((r, c))

    for color, queens in color_groups.items():
        if len(queens) > 1:  # More than one queen for a color
            print(f"Invalid state: More than one queen in color group '{color}'. Queens: {queens}")
            return False

    # Rule 4: No rows, columns, or color groups with only zeros
    for r in range(rows): #rows
        if None not in temp_grid[r] and all(cell == 0 for cell in temp_grid[r]):
            print(f"Invalid state: Row {r} has only 0s and no None values. Row: {temp_grid[r]}")
            return False

    for c in range(cols): #column
        column = [temp_grid[r][c] for r in range(rows)]
        if None not in column and all(cell == 0 for cell in column):
            print(f"Invalid state: Column {c} has only 0s and no None values. Column: {column}")
            return False

    for color in color_groups: #colors
        color_cells = [
            temp_grid[r][c]
            for r in range(rows)
            for c in range(cols)
            if grid_colors[r][c] == color
        ]
        if None not in color_cells and all(cell == 0 for cell in color_cells):
            print(f"Invalid state: Color group '{color}' has only 0s and no None values. Cells: {color_cells}")
            return False

    # Rule 5: No adjacent queens
    adjacent_directions = [
        (-1, -1), (-1, 0), (-1, 1),
        (0, -1),           (0, 1),
        (1, -1), (1, 0), (1, 1),
    ]
    for r in range(rows):
        for c in range(cols):
            if temp_grid[r][c] == 1:
                for dr, dc in adjacent_directions:
                    nr, nc = r + dr, c + dc
                    if 0 <= nr < rows and 0 <= nc < cols and temp_grid[nr][nc] == 1:
                        print(f"Invalid state: Adjacent queens found at ({r}, {c}) and ({nr}, {nc}).")
                        return False

    return True

def is_goal_state(temp_grid):
    """Check if the current grid satisfies all constraints."""
    return all(cell is not None for row in temp_grid for cell in row)

def print_temp_grid(temp_grid):
    """Print the temp grid in a readable format."""
    for row in temp_grid:
        print(" ".join(str(cell) if cell is not None else "." for cell in row))


def print_colored_grid(temp_grid, grid_colors):
  
    # color mapping
    color_mapping = {
        '0': '\033[95m',        # Purple
        '1': '\033[33m',        # Orange
        '2': '\033[94m',        # Blue
        '3': '\033[92m',        # Green
        '4': '\033[97m',        # White/Grey
        '5': '\033[91m',        # Red
        '6': '\033[38;5;226m',  # Bright Yellow
        '7': '\033[38;5;16m',   # Dark Brown
        '8': '\033[38;5;165m',  # Pink
        '9': '\033[96m',        # Cyan
        '10': '\033[38;5;46m',  # Bright Green
        '11': '\033[38;5;51m',  # Bright Blue
        '12': '\033[38;5;208m', # Dark Orange
        '13': '\033[38;5;27m',  # Deep Blue
        '14': '\033[38;5;129m', # Magenta
        '15': '\033[38;5;202m', # Bright Orange
    }
    reset_color = '\033[0m'

    rows, cols = len(temp_grid), len(temp_grid[0])

    for r in range(rows):
        row_display = []
        for c in range(cols):
            value = temp_grid[r][c]
            color = grid_colors[r][c]

            if value == 1:
                cell = f"{color_mapping[color]}X{reset_color}"
            elif value == 0:
                cell = f"{color_mapping[color]}-{reset_color}"
            else:
                cell = f"{color_mapping[color]}.{reset_color}"

            row_display.append(cell)

        print(" ".join(row_display))

def find_most_constrained_cells(temp_grid, grid_colors):
    """
    Find the most constrained cells based on rows, columns, and colors.
    Returns a list of the most constrained cells for rows, columns, or color groups
    that do not yet have a queen.
    """
    rows, cols = len(temp_grid), len(temp_grid[0])
    
    # Count free cells per row, column, and color, excluding groups that already have a queen
    row_free_counts = [
        sum(1 for cell in temp_grid[r] if cell is None) if not any(cell == 1 for cell in temp_grid[r]) else float('inf')
        for r in range(rows)
    ]
    col_free_counts = [
        sum(1 for r in range(rows) if temp_grid[r][c] is None) if not any(temp_grid[r][c] == 1 for r in range(rows)) else float('inf')
        for c in range(cols)
    ]
    color_free_counts = {}
    color_has_queen = set()

    for r in range(rows):
        for c in range(cols):
            color = grid_colors[r][c]
            if temp_grid[r][c] == 1:
                color_has_queen.add(color)
            if temp_grid[r][c] is None:
                color_free_counts[color] = color_free_counts.get(color, 0) + 1

    # Exclude color groups that already have a queen
    for color in color_has_queen:
        if color in color_free_counts:
            color_free_counts[color] = float('inf')

    # Find the minimum free cells
    min_row_free = min(row_free_counts)
    min_col_free = min(col_free_counts)
    min_color_free = min(color_free_counts.values()) if color_free_counts else float('inf')

    # Determine the type with the fewest free cells
    min_free = min(min_row_free, min_col_free, min_color_free)
    most_constrained_cells = []

    if min_row_free == min_free:
        for r in range(rows):
            if row_free_counts[r] == min_row_free:
                most_constrained_cells.extend([(r, c) for c in range(cols) if temp_grid[r][c] is None])

    if min_col_free == min_free:
        for c in range(cols):
            if col_free_counts[c] == min_col_free:
                most_constrained_cells.extend([(r, c) for r in range(rows) if temp_grid[r][c] is None])

    if min_color_free == min_free:
        for r in range(rows):
            for c in range(cols):
                color = grid_colors[r][c]
                if temp_grid[r][c] is None and color_free_counts[color] == min_color_free:
                    most_constrained_cells.append((r, c))

    # Remove duplicates and return
    most_constrained_cells = list(set(most_constrained_cells))
    return most_constrained_cells


def place_queens_with_selenium(driver, solution_grid):
 
    rows = len(solution_grid)
    cols = len(solution_grid[0])

    for r in range(rows):
        for c in range(cols):
            if solution_grid[r][c] == 1:  # We need to place a queen here
                cell_selector = f"div[data-cell-idx='{r * cols + c}']"
                cell_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, cell_selector))
                )

                try:
                    # Double-click the cell to set a Queen
                    cell_element.click()
                    cell_element.click()
                except Exception as e:
                    print(f"Error placing queen at ({r}, {c}): {e}")
                

# Main Script
if __name__ == "__main__":
    # File path
    file_path = '/Users/janverwey/Desktop/Queens Content/LinkedIn_Queens_236.txt'

    # Read the file
    with open(file_path, 'r') as file:
        html_content = file.read()

    # Parse the HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract grid dimensions
    grid = soup.find('div', id='queens-grid')
    rows = int(grid['style'].split('--rows: ')[1].split(';')[0])
    cols = int(grid['style'].split('--cols: ')[1].split(';')[0])

    # Extract cell information
    cells = grid.find_all('div', class_='queens-cell-with-border')

    # Initialize the grid
    temp_grid = [[None for _ in range(cols)] for _ in range(rows)]
    grid_colors = [[None for _ in range(cols)] for _ in range(rows)]

    for cell in cells:
        data_idx = int(cell['data-cell-idx'])
        cell_color = cell.get('class', [])
        color = None
        for class_name in cell_color:
            if 'cell-color-' in class_name:
                color = class_name.split('-')[-1]
                break

        # Determine row and column
        row = data_idx // cols
        col = data_idx % cols

        # Fill the grid colors
        grid_colors[row][col] = color

    # Dummy variables for compatibility
    variables = [[f"x{r+1}{c+1}" for c in range(cols)] for r in range(rows)]

    # Solve the puzzle
    solution = solve_with_backtracking_and_propagation(temp_grid, grid_colors, variables)

    # Use the print_colored_grid function after solving the puzzle
    if solution:
        print("Solution found:")
        print_colored_grid(solution, grid_colors)
    else:
        print("No solution found.")